#### Scenario


 2. 기준 인풋 데이터를 토대로(평가) 관광지에 대한 평가 평균값을 계산.
3. Top 레이팅을 평가.
------------------------------------------
탐색
1. 이름/탐색 기준으로 key 값 지정
2. 평균값 내림차순으로 Key값에 의한 sort
3. 원하는 지역 탐색시 도메인 + 쿼리에 검색내용 추가 : 링크로 이동할 수 있게끔 도와줌
4. 

              

###### 1. Program begins

### 2. RECORD

2-0 ALL IMPORT LIBRARY

In [81]:
import requests
# !pip install haversine
# !pip install tabulate[widechars]
import haversine
import datetime
from dateutil.parser import parse
import pandas as pd
from ast import literal_eval
import time
from tabulate import tabulate

2-1 importing classes

2-1-1 Find date class

In [82]:
class Find:
    def fr_date(self,p_date): #Record
        from dateutil.parser import parse
        p_date= parse(p_date)
        p_date = p_date.strftime('%Y-%m-%d-%a')
        return p_date
    
    def fs_date(self,s_date): #Searching
        from dateutil.parser import parse
        s_date = parse(s_date)
        s_date = s_date.strftime('%Y-%m-%d-%a')
        return s_date

2-1-2 Cal_Distance class

In [83]:
class Cal_Distance:
    def getLatLng(self,p_start,p_dn_name):
        import requests
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_s_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_s_Lat = float(p_s_places[0]['y'])
        p_s_Lng = float(p_s_places[0]['x'])
        p_s_LatLng=(p_s_Lat,p_s_Lng)

        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_dn_name)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_d_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_d_Lat = float(p_d_places[0]['y'])
        p_d_Lng = float(p_d_places[0]['x'])
        p_d_LatLng=(p_d_Lat,p_d_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
        return r_s_d

    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    

2-1-3 Evaluation Class

In [84]:
#  기준값적용 (평가기준값)
#  - 기준값 지정 (총 5점)
#  - 평가값을 받는 것 = 이동수단 / 비용 / 거리 ** 서울역 기준 /   

class Evaluation:
    def evalue_costs(self,p_cost,p_length_of_stay): #관광데이터랩 1회 평균 국내여행지출 금액 133,000원 정규분포도 활용
        if p_cost/p_length_of_stay <= 50000: #50000
            return 5
        elif 50000 < p_cost/p_length_of_stay <= 100000: #50001~100000
            return 4
        elif 100000 < p_cost/p_length_of_stay <= 150000: #100001~150000
            return 3
        elif 150000 < p_cost/p_length_of_stay <= 200000: #150001~200000
            return 2
        else: #200001이상.
            return 1
        
    def evalue_by_distance(self,p_transportation,r_s_d): #이동수단을 직선거리로 평가.
        if p_transportation == "도보":
            if r_s_d <= 1:
                return 5
            elif 1 < r_s_d <= 2:
                return 4
            elif 2 < r_s_d <= 3:
                return 3
            elif 3 < r_s_d <= 4:
                return 2
            else:
                return 1
            
        elif p_transportation == "지역버스" or p_transportation == "지하철" :
            if r_s_d <= 10:
                return 5
            elif 10 < r_s_d <= 20:
                return 4
            elif 20 < r_s_d <= 30:
                return 3
            elif 30 < r_s_d <= 40:
                return 2
            else:
                return 1
            
        elif p_transportation == "자차" or p_transportation == "자동차":
            if r_s_d <= 50:
                return 5
            elif 50 < r_s_d <= 100:
                return 4
            elif 100 < r_s_d <= 150:
                return 3
            elif 150 < r_s_d <= 200:
                return 2
            else:
                return 1
            
        elif p_transportation == "고속버스":
            if r_s_d <= 70:
                return 5
            elif r_s_d <= 140:
                return 4
            elif r_s_d <= 210:
                return 3
            elif r_s_d <= 280:
                return 2
            else:
                return 1
            
        elif p_transportation == "전철" or p_transportation == "KTX":
            if r_s_d <= 90:
                return 1
            elif 90 < r_s_d <= 180:
                return 2
            elif 180 < r_s_d <= 270:
                return 3
            elif 270 < r_s_d <= 360:
                return 4
            else:
                return 5
            
        elif p_transportation == "비행기":
            if r_s_d <= 120:
                return 1
            elif 120 < r_s_d <= 200:
                return 2
            elif 200 < r_s_d <= 280:
                return 3
            elif 280< r_s_d <= 360:
                return 4
            else:
                return 5
        
        elif p_transportation == "비행기" and p_start == "제주도":
            return 5
        
    def evalue_by_satisfaction(self,p_satisfaction): #관광만족도 개인 평가 
        if p_satisfaction <= 5:
            return p_satisfaction
        else:
            return 5
    
            
        

2-4 Record Class

2-4-1 Method(Evaluation,Cal_Distance,Find) class 상속

In [85]:
class Method(Evaluation,Cal_Distance,Find):
    pass

2-5 Data Record class

In [86]:
class Data_Record:
    def first_record(self):
        with open("data_repository.txt","w") as data:   
            print("첫번째 기록입니다.")
            while True:
                #Method 클래스 지정. 
                m = Method()
                #입력 데이터값 
                print('\n')
                p_date = input("여행날짜 ex)20210701 : ")
                f_p_date = m.fr_date(p_date)
                
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/등 : ")
                p_start = input("출발 장소를 입력하세요 : ")
                p_dn_name = input("여행목적지(관광지명) : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요. 도보/지역버스(지하철)/자동차/고속버스/전철(KTX)/비행기 : ")
                p_length_of_stay = int(input("총 여행 기간 : __ 일 : "))
                p_cost = int(input("총 지출 금액 : __ 원 : "))
                p_satisfaction = float(input("관광만족도는 어떠셧습니까? <매우좋았음:5 ~ 비추천:1> : "))
                print('\n\n')
                input_data = {'여행날짜':f_p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지명':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction}      
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                input_value = {'지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                
                #데이터 기록 값
                past_data = {'여행날짜':p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지명':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction,
                             '지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                print("기록하신 내용입니다.")
                print(input_data)
                print('\n')
                print("기록한 내용에 대한 평가입니다.")
                print(input_value)
                
                correct = input("이 기록이 맞으시면 '예', 아니면 '아니오' 을 눌러주세요 : ")
                correct
                if correct == "예":
                    print('\n')
                    print("기록하신 내용이 저장됩니다.")
                    past_data = str(past_data)
                    data.write(past_data +'\n')
                else:
                    print('\n')
                    print("기록하신 내용이 삭제됩니다.")
                
                Question_end = input("여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                elif Question_end == "아니오":
                    while True:
                        End_Q = input("종료하시겠습니까? 예/아니오 : ")
                        if End_Q == "예":
                            print("기록이 종료되었습니다.")
                            print('\n')
                            print("추가 기록된 내용입니다.")
                            print(past_data)
                            break
                        elif End_Q == "아니오":
                            Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                            break
                        else:
                            print("잘못 입력하셨습니다.")
                            continue
                            
        with open("data_repository.txt",'r') as data_r: data_r = data_r.read()
        return print(data_r,sep='\n')
                    
                        
                    
    def another_record(self):
        with open("data_repository.txt","a") as data:    
            print("기존 데이터에 기록합니다.")
            while True:
                #Method 클래스 지정
                m = Method()
                #입력 데이터값 
                print('\n')
                p_date = input("여행날짜 ex)20210701 : ")
                f_p_date = m.fr_date(p_date)
                print(f_p_date)
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/등 : ")
                p_start = input("출발 장소를 입력하세요 : ")
                p_dn_name = input("여행목적지(관광지명) : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요.(택1) 도보/지역버스(지하철)/자동차/고속버스/전철(KTX)/비행기 : ")
                p_length_of_stay = int(input("총 여행 기간 : __ 일 : "))
                p_cost = int(input("총 지출 금액 : __ 원 : "))
                p_satisfaction = float(input("관광만족도는 어떠셧습니까? <매우좋았음:5 ~ 비추천:1> : "))
                print('\n\n')
                input_data = {'여행날짜':f_p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지명':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction}
                
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                #데이터 기록 값
                past_data = {'여행날짜':f_p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지명':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction,
                             '지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                input_value = {'지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
            
                print("기록하신 내용입니다.")
                print(input_data)
                print('\n')
                
                print("기록한 내용에 대한 평가입니다.")
                print(input_value)
                
                correct = input("이 기록이 맞으시면 '예', 아니면 '아니오' 을 눌러주세요 : ")
                correct
                if correct == "예":
                    print('\n')
                    print("기록하신 내용이 저장됩니다.")
                    print('\n')
                    past_data = str(past_data)
                    data.write(past_data +'\n')
                else:
                    print('\n')
                    print("기록하신 내용이 삭제됩니다.")
                    
                Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                elif Question_end == "아니오":
                    while True:
                        End_Q = input("종료하시겠습니까? 예/아니오 : ")
                        if End_Q == "예":
                            print("기록이 종료되었습니다.")
                            print('\n')
                            print("추가 기록된 내용입니다.")
                            print(past_data)
                            break
                        elif End_Q == "아니오":
                            Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                            break
                        else:
                            print("잘못 입력하셨습니다.")
                            continue
                break


# trial

In [87]:
class Record_Program:
    def start_program(self):
        r = Data_Record()
        print("기록 프로그램이 실행됩니다.")
        print('\n')
        print("-----------------------------------------------------------------------")
        print("""기록 프로그램은 사용자의 여행기록을 기록하고 평가합니다.\n평가기준은 다음과 같습니다.""")
        print("""평가기준 1 : 출발지부터 목적지까지의 직선거리에 대한 이동수단별 평가\n평가기준 2 : 여행 1일 동안 사용한 비용에 대한 평가\n평가기준 3 : 여행만족도에 대한 주관적 평가""")
        print("""평가된 데이터는 관광기록 순위 혹은 탐색에 이용됩니다.\n많은 이용 부탁드립니다. -김광훈""")
        print("-----------------------------------------------------------------------")
        time.sleep(0.5)
        Question = input("처음 기록합니까?  예/아니오 : ")
        if Question == "예":
            print('\n')
            print("================<중요>================")
            print("처음 기록하신다면 기존 기록은 사라집니다.")
            print("======================================")
            om_Question = input("""기존 데이터 삭제 후, 처음부터 기록하시려면 <기록>를 작성해주세요.\n원치 않으시면 <스페이스 한 번> 입력해주세요 : """)
            if om_Question == "기록":
                r.first_record()
            elif om_Question == " ":
                while True:
                    om2_Question = input("""처음으로 돌아가시려면 <처음>을, 기존 데이터 삭제 후, 처음부터 기록하시려면 <기록>을 작성해주세요.\n종료를 원하시면 <종료>를 입력하세요. : """)
                    if om2_Question == "기록":
                        print('\n')
                        r.first_record()
                        break
                    elif om2_Question == "처음":
                        print('\n\n')
                        Record_Program.start_program(self)
                        break
                    elif om2_Question == "종료":
                        print("-----------------------------------------------------------------------")
                        print('기록 프로그램을 종료합니다')
                        print("프로그램 시작단계로 돌아갑니다")
                        print('\n\n')
                        MP = MyfirstProgram
                        MP.start(self)
                        break
                    else:
                        continue
                
        else:
            time.sleep(0.2)
            Another_Question = input("기존 데이터에 기록하겠습니까? 예/아니오 : ")
    
            if Another_Question == "예":
                r.another_record()
                
            elif Another_Question == "아니오":
                print("<아니오>를 입력하셨습니다.")
                while True:
                    oma_Question = input("처음으로 돌아가시려면 <처음>을, 종료하시려면 <종료>를 입력하세요. : ")
                    if oma_Question == "처음":
                        Record_Program.start_program(self)
                        break
                    elif oma_Question == '종료':
                        print("-----------------------------------------------------------------------")
                        print('기록 프로그램을 종료합니다')
                        print("프로그램 시작단계로 돌아갑니다")
                        print('\n\n')
                        MP = MyfirstProgram
                        MP.start(self)
                    
                        break
                    else:
                        print("잘못 입력하셨습니다.")
                        continue   
                        
            else:
                print("-----------------------------------------------------------------------")
                print('기록 프로그램을 종료합니다')

In [92]:
t = Record_Program()
t.start_program()

기록 프로그램이 실행됩니다.


-----------------------------------------------------------------------
기록 프로그램은 사용자의 여행기록을 기록하고 평가합니다.
평가기준은 다음과 같습니다.
평가기준 1 : 출발지부터 목적지까지의 직선거리에 대한 이동수단별 평가
평가기준 2 : 여행 1일 동안 사용한 비용에 대한 평가
평가기준 3 : 여행만족도에 대한 주관적 평가
평가된 데이터는 관광기록 순위 혹은 탐색에 이용됩니다.
많은 이용 부탁드립니다. -김광훈
-----------------------------------------------------------------------
처음 기록합니까?  예/아니오 : 아니오
기존 데이터에 기록하겠습니까? 예/아니오 : 아니오
<아니오>를 입력하셨습니다.
처음으로 돌아가시려면 <처음>을, 종료하시려면 <종료>를 입력하세요. : 종료
-----------------------------------------------------------------------
기록 프로그램을 종료합니다
프로그램 시작단계로 돌아갑니다



첫 프로그램을 실행해주셔서 감사합니다.
이 프로그램은 사용자의 '국내관광'의 기록을 기록하고 평가하는 프로그램과
지금까지 기록했던 데이터를 탐색하는 프로그램입니다.
데이터의 수가 많아질 수록 더욱 더 신뢰도 높은 정보를 제공할 수 있습니다.
사용해주셔서 감사합니다.


-----------------------------------------------------------------------
평가 순위 별 Top 10 목록입니다.
아직까지 구현이 완벽하게 되지 않아, 표본수와 평가순위의 논리성이 많이 부족합니다
앞으로 더 개선해 나아갈 예정입니다. 감사합니다.
-----------------------------------------------------------------------


Top10
------------------------

### 3. Loading Data Class

In [93]:
class Loading:
    def load_data(self):
        with open('data_repository.txt') as f:
            data = f.readlines()
            data = [literal_eval(x.strip()) for x in data]   
            data = pd.DataFrame(data)
            data['평가점수총합'] = data['지출비용평가점수'] + data['이동비율평가점수'] + data['만족도평가점수']
            data['평균점수'] = data['평가점수총합'] / 3
            data = data.round({'평균점수':2})
            data['평가순위'] = data['평균점수'].rank(ascending=False,method='dense')
            data['평가순위'] = data['평가순위'].astype(int)
            data = data.sort_values(by='평가순위')
            data = data.to_csv("data")
            data = pd.read_csv("data")
            data.reset_index(drop=True)
            data
    
    def len_row_data(self):
        data = pd.read_csv("data")
        Value1 = data["관광유형"]
        Value2 = data["관광지명"]
        Value12 = data[Value1 & Value2]
        data['여행누적횟수'] = len(Value12.index)
        return data
    
    def top10(self):
        data = pd.read_csv("data")
        data.reset_index(drop=True)
        top_10 = data[['평가순위','관광지명','평균점수']]
        top_10.to_csv("top10")
        top_list = pd.read_csv("top10",index_col=0)
        top_list = top_list.set_index('평가순위')
        top_list.to_csv("top10")
        top = pd.read_csv('top10',index_col=0)
        print(tabulate(top.head(10),headers='keys'))


# 4. Searching Class

In [ ]:
class Searching_data:
    def searching_data(self):
        print("데이터를 불러오는 중입니다.")
        #Method 클래스 지정
        m = Method()
        time.sleep(1)
        print("\n")
        print("탐색 프로그램이 실행됩니다.")
        print('\n')
        print("-----------------------------------------------------------------------")
        print("""탐색 프로그램은 사용자의 여행기록을 탐색합니다.""")
        print("""탐색요소는 목록에서 결정할 수 있습니다.\n세부요소가 가능한 목록 또한 있습니다.""")
        print("""탐색 프로그램은 기록된 데이터를 기반으로 합니다.\n따라 많은 자료를 필요로합니다.""")
        print("""따라서 많은 기록과 많은 이용 부탁드립니다. -김광훈""")
        print("-----------------------------------------------------------------------")
        L = Loading()
        L.load_data()
        time.sleep(0.2)
        while True:
            Question = input("탐색하시겠습니까?  예/아니오 : ")
            if Question == "예":
                print('\n')
                print("탐색 목록이 제공됩니다.")
                print("\n")
                print("========================================<목록>========================================")
                print("1) 여행날짜 2) 관광유형 3) 출발지 4) 관광지명 5) 이동수단 6) 관광기간 7) 총비용 8) 만족도")
                print("======================================================================================")
                s_Question = input("탐색하고 싶은 목록값을 입력하세요 :  ")
                if s_Question == "여행날짜":
                    s_date = input("탐색하실 날짜를 입력하세요 ex)20210701 : ")
                    f_s_date = m.fs_date(s_date)
                    index = data["여행날짜"] == f_s_date
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['평가순위','출발지','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(["여행날짜","관광유형","관광지명"]).mean()
                    Answer = Answer.sort_values(ascending = False, by="평균점수")
                    print("\n")
                    print("-----------------------------------------------------------------------")
                    print(tabulate(Answer.head(10),headers=["여행날짜","관광유형","관광지명"]))
                    print("-----------------------------------------------------------------------")

                elif s_Question == "관광유형":
                    s_type = input("탐색하실 유형을 입력하세요 : ")
                    index = data["관광유형"] == s_type
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['평가순위','출발지','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(['여행날짜', "관광유형","관광지명"]).mean()
                    detail_Q = input("세부적으로 검색하시겠습니까? 예/아니오 : ")

                    if detail_Q == "예":
                        s_detail = input("탐색하실 관광지명을 입력하세요 : ")
                        details = data["관광지명"] == s_detail
                        Find_data = data[index & details]
                        Find_data = Find_data.reset_index()
                        Find_data = Find_data.drop(['평가순위','출발지','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                        Answer_d = Find_data.groupby(["관광유형","관광지명"]).mean()
                        Answer_d = Answer_d.sort_values(ascending = False, by="평균점수")
                        print("\n")
                        print("-----------------------------------------------------------------------")
                        print(tabulate(Answer_d.head(10),headers=["관광유형","관광지명"]))
                        print("-----------------------------------------------------------------------")
                    elif detail_Q == "아니오":
                        Answer = Answer.sort_values(ascending = False, by="평균점수")
                        print("\n")
                        print("-----------------------------------------------------------------------")
                        print(tabulate(Answer.head(10),headers=["관광유형","관광지명"]))
                        print("-----------------------------------------------------------------------")

                elif s_Question == "출발지":
                    st_type = input("탐색하실 출발지를 입력하세요 : ")
                    index =  data["출발지"] == st_type 
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['평가순위','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(['출발지',"관광유형","관광지명"]).mean()
                    Answer = Answer.sort_values(ascending = False, by="평균점수")
                    print("\n")
                    print("-----------------------------------------------------------------------")
                    print(tabulate(Answer.head(10),headers=(['출발지',"관광유형","관광지명"]))
                    print("-----------------------------------------------------------------------")

                elif s_Question == "관광지명":
                    st_type = input("탐색하실 관광지명을 입력하세요 : ")
                    index =  data["관광지명"] == st_type 
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['출발지','평가순위','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(["관광지명","관광유형"]).mean()
                    detail_Q = input("세부적으로 검색하시겠습니까? 예/아니오 : ")

                    if detail_Q == "예":
                        s_detail = input("탐색하실 관광유형 입력하세요 : ")
                        details = data["관광유형"] == s_detail
                        Find_data = data[index & details]
                        Find_data = Find_data.reset_index()
                        Find_data = Find_data.drop(['평가순위','출발지','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                        Answer_d = Find_data.groupby(["관광지명","관광유형"]).mean()
                        Answer_d = Answer.sort_values(ascending = False, by="평균점수")
                        print("\n")
                        print("-----------------------------------------------------------------------")
                        print(tabulate(Answer_d.head(10),headers=["관광지명","관광유형"]))
                        print("-----------------------------------------------------------------------")
                    elif detail_Q == "아니오":
                        Answer = Answer.sort_values(ascending = False, by="평균점수")
                        print("\n")
                        print("-----------------------------------------------------------------------")
                        print(tabulate(Answer.head(10),headers=["관광지명","관광유형"]))
                        print("-----------------------------------------------------------------------")



                elif s_Question == "이동수단":
                    st_type = input("탐색하실 이동수단을 입력하세요 : ")
                    index =  data["이동수단"] == st_type 
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['출발지','평가순위','관광기간','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(['이동수단',"관광유형","관광지명"]).mean()
                    Answer = Answer.sort_values(ascending = False, by="평균점수")
                    print("\n")
                    print("-----------------------------------------------------------------------")
                    print(tabulate(Answer.head(10),headers=['이동수단',"관광유형","관광지명"]))
                    print("-----------------------------------------------------------------------")


                elif s_Question == "관광기간":
                    st_type = input("탐색하실 관광기간을 입력하세요 : ")
                    index =  data["관광기간"] == int(st_type)
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['이동수단','출발지','평가순위','총비용','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(['관광기간',"관광유형","관광지명"]).mean()
                    Answer = Answer.sort_values(ascending = False, by="평균점수")
                    print("\n")
                    print("-----------------------------------------------------------------------")
                    print(tabulate(Answer.head(10),headers=['관광기간',"관광유형","관광지명"]))
                    print("-----------------------------------------------------------------------")

                elif s_Question == "총비용":
                    st_type = int(input("탐색하실 최대 총비용을 입력하세요 : "))
                    data["총비용"] = data["총비용"].astype(int)
                    index =  data["총비용"] <= st_type
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['이동수단','출발지','평가순위','관광기간','만족도','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(['총비용',"관광유형","관광지명"]).mean()
                    Answer = Answer.sort_values(ascending = False, by="평균점수")
                    print("\n")
                    print("-----------------------------------------------------------------------")
                    print(tabulate(Answer.head(10),headers=['총비용',"관광유형","관광지명"]))
                    print("-----------------------------------------------------------------------")

                elif s_Question == "만족도":
                    st_type = int(input("탐색하실 만족도를 입력하세요 : "))
                    data["만족도"] = data["만족도"].astype(int)
                    index =  data["만족도"] <= st_type
                    Find_data = data[index]
                    Find_data = Find_data.reset_index()
                    Find_data = Find_data.drop(['이동수단','출발지','평가순위','관광기간','총비용','지출비용평가점수',
                          '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
                    Answer = Find_data.groupby(['만족도',"관광유형","관광지명"]).mean()
                    Answer = Answer.sort_values(ascending = False, by="평균점수")
                    print("\n")
                    print("-----------------------------------------------------------------------")
                    print(tabulate(Answer.head(10),headers=['만족도',"관광유형","관광지명"]))
                    print("-----------------------------------------------------------------------")
                    
            elif Question  == "아니오":
                print("<아니오>를 입력하셨습니다.")
                while True:
                    End_Question = input("탐색을 종료하시겠습니까? 예 / 아니오 : ")
                    if End_Question == "예":
                        print("프로그램 처음으로 돌아갑니다.")
                        print('\n\n')
                        MP = MyfirstProgram
                        MP.start(self)
                        break
                    elif End_Question == "아니오":
                        print("탐색 프로그램 처음부터 진행합니다.")
                        Searching_data.searching_data(self)
                        break
                    else:
                        print("잘못 입력하셨습니다.")
                        continue





In [ ]:
CC = Searching_data()
CC.searching_data()

# LAST PROGRAM ALL CLASS

In [55]:
class MyfirstProgram:
    def start(self):
        L = Loading()
        print("첫 프로그램을 실행해주셔서 감사합니다.")
        time.sleep(0.5)
        print("이 프로그램은 사용자의 '국내관광'의 기록을 기록하고 평가하는 프로그램과\n지금까지 기록했던 데이터를 탐색하는 프로그램입니다.")
        time.sleep(0.5)
        print('데이터의 수가 많아질 수록 더욱 더 신뢰도 높은 정보를 제공할 수 있습니다.')
        time.sleep(0.5)
        print('사용해주셔서 감사합니다.')
        print('\n')
        time.sleep(0.5)
        print("-----------------------------------------------------------------------")
        print("평가 순위 별 Top 10 목록입니다.")
        print("아직까지 구현이 완벽하게 되지 않아, 표본수와 평가순위의 논리성이 많이 부족합니다")
        print("앞으로 더 개선해 나아갈 예정입니다. 감사합니다.")
        print("-----------------------------------------------------------------------")
        time.sleep(0.5)
        print("\n")
        print("Top10")
        print("-----------------------------------------------------------------------")
        L.load_data()
        L.top10()
        print("-----------------------------------------------------------------------")
        print("\n\n")
        time.sleep(0.5)
        print("기록 및 평가 프로그램과 탐색 프로그램이 있습니다.")
        while True:
            PS_Q = input("프로그램을 시작하시겠습니까? 시작하려면 <예> , 종료하려면 <아니오>를 입력하세요 : ")
            if PS_Q == "예":
                while True:
                    P_Question = input("원하시는 프로그램을 입력해주세요. <기록> / <탐색> : ") 

                    if P_Question == "기록":
                        CR = Record_Program()
                        CR.start_program()

                    elif P_Question == "탐색":
                        SD = Searching_data()
                        SD.searching_data()

                    else:
                        print("잘못 입력하셨습니다.")
                        continue

            elif PS_Q == "아니오":
                print("\n")
                print("이용해주셔서 정말 감사합니다.")
                print("\n")
                print("Program End")
                print("-----------------------------------------------------------------------")
                break

            else:
                print("잘못 입력하셨습니다.")
                continue





        
        
        

In [56]:
MP = MyfirstProgram()
MP.start()

첫 프로그램을 실행해주셔서 감사합니다.
이 프로그램은 사용자의 '국내관광'의 기록을 기록하고 평가하는 프로그램과
지금까지 기록했던 데이터를 탐색하는 프로그램입니다.
데이터의 수가 많아질 수록 더욱 더 신뢰도 높은 정보를 제공할 수 있습니다.
사용해주셔서 감사합니다.


-----------------------------------------------------------------------
평가 순위 별 Top 10 목록입니다.
아직까지 구현이 완벽하게 되지 않아, 표본수와 평가순위의 논리성이 많이 부족합니다
앞으로 더 개선해 나아갈 예정입니다. 감사합니다.
-----------------------------------------------------------------------


Top10
-----------------------------------------------------------------------
  평가순위  관광지명                평균점수
----------  --------------------  ----------
         1  여의나루한강공원            5
         2  경주                        4.67
         3  경주                        4.33
         3  대부도                      4.33
         4  홍천비발디파크              4.27
         5  을왕리                      4
         5  제주도                      4
         5  부산                        4
         5  동해시종합버스터미널        4
         6  지리산                      3.9
------------------------------------------------------------

KeyboardInterrupt: Interrupted by user

In [24]:
S = Searching_data()
S.searching_data()

데이터를 불러오는 중입니다.


탐색을 시작합니다.


========================================<목록>========================================
1) 여행날짜 2) 관광유형 3) 출발지 4) 관광지명 5) 이동수단 6) 관광기간 7) 총비용 8) 만족도
탐색하고 싶은 목록값을 입력하세요 :  만족도
탐색하실 만족도를 입력하세요 : 5


,,,,평균점수
여행날짜,만족도,관광유형,관광지명,
20210704,5,캠핑,여의나루한강공원,5.00
2020-10-22-Thu,5,도시,경주,4.67
20210130,5,식사,경주,4.33
20210220,4,술,대부도,4.33
20201211,4,스키,홍천비발디파크,4.27
20210406,5,도시,제주도,4.00
20210101,3,바다,을왕리,4.00
20191011,4,도시,부산,4.00
20210603,5,바다,동해시종합버스터미널,4.00


3-2. Extract the data list with Key : Value

###### Code로 변환 -- 알고만 있자

In [270]:
# !pip install plotly
# import plotly.express as px
# Seoul = data['관광지이름'] == '서울역'
# find_Seoul = data[Seoul]
# find_Seoul

In [272]:
# data['평가순위'] = data['평균점수'].rank(ascending=False,method='dense')
# data = data.sort_values(ascending=True,by = "평가순위")
# data = data["평가순위"].round(decimals=0)
# data

3-5 get sorted by Key : Value

In [50]:
with open('data_repository.txt') as f:
    data = f.readlines()
data = [literal_eval(x.strip()) for x in data]   
data = pd.DataFrame(data)
data['평가점수총합'] = data['지출비용평가점수'] + data['이동비율평가점수'] + data['만족도평가점수']
data['평균점수'] = data['평가점수총합'] / 3
data = data.round({'평균점수':2})
# data['평가순위'] = data['평균점수'].rank(ascending=False,method='dense')
# data['평가순위'] = data['평가순위'].astype(int)
# data = data.sort_values(by='평가순위')
# data = data.set_index('평가순위',drop=True,append=False)
# data = data.to_csv("data")
data = pd.read_csv("data")
data
# data.reset_index(drop=True)
# top_10 = data[['평가순위','관광유형','관광지이름','평균점수']]
# top_10.to_csv("top10")
# top_list = pd.read_csv("top10",index_col=0)
# top_list = top_list.set_index('평가순위')
# top_list.to_csv("top10")
# top = pd.read_csv('top10',index_col=0)
# top.head(10)

,평가순위,여행날짜,관광유형,출발지,관광지이름,이동수단,관광기간,총비용,만족도,지출비용평가점수,이동비율평가점수,만족도평가점수,평가점수총합,평균점수
0,1,20210704,캠핑,여의나루역,여의나루한강공원,도보,1,50000,5.0,5,5,5.0,15.0,5.00
1,2,2020-10-22-Thu,도시,걸포1로 10,경주,KTX,2,100000,5.0,5,4,5.0,14.0,4.67
2,3,20210130,식사,개포동,경주,KTX,3,150000,5.0,5,3,5.0,13.0,4.33
3,3,20210220,술,금천구독산동,대부도,자동차,2,150000,4.0,4,5,4.0,13.0,4.33
4,4,20201211,스키,금천구독산동,홍천비발디파크,자동차,2,180000,4.8,4,4,4.8,12.8,4.27
5,5,20210603,바다,서울고속버스터미널,동해시종합버스터미널,고속버스,2,150000,5.0,4,3,5.0,12.0,4.00
6,5,20191011,도시,영등포,부산,비행기,3,300000,4.0,4,4,4.0,12.0,4.00
7,5,20210101,바다,서울역,을왕리,고속버스,3,200000,3.0,4,5,3.0,12.0,4.00
8,5,20210406,도시,인천공항,제주도,비행기,3,600000,5.0,2,5,5.0,12.0,4.00
9,6,20210605,휴가,서산시해미면기지리,지리산,자동차,2,100000,4.7,5,2,4.7,11.7,3.90


class Cal_Distance:
    def getLatLng(self,p_start,p_dn_name):
        import requests
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_s_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_s_Lat = float(p_s_places[0]['y'])
        p_s_Lng = float(p_s_places[0]['x'])
        p_s_LatLng=(p_s_Lat,p_s_Lng)

        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_dn_name)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_d_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_d_Lat = float(p_d_places[0]['y'])
        p_d_Lng = float(p_d_places[0]['x'])
        p_d_LatLng=(p_d_Lat,p_d_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
        return r_s_d

    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    